<a href="https://colab.research.google.com/github/ushifali/final-23/blob/master/Collaborative_Filtering_Attractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd

# Load user preferences data
user_preferences_df  = pd.read_csv('/content/drive/MyDrive/Filtering_Data/user_preferences_attractions.csv', encoding='utf-8')

# Load attraction data
attractions_df = pd.read_csv('/content/drive/MyDrive/Filtering_Data/content_filtered_data_attraction.csv', encoding='utf-8')
attractions_df.head()

,Unnamed: 0.1,Unnamed: 0,address,city,description,latitude,longitude,price,state,street,title,totalScore,categories,image_url,Distance
0,274,274,"Murugeshpalya, Jeevan Bima Nagar, Bengaluru, K...",Bengaluru,NaN,12.961314,77.656118,NaN,Karnataka,NaN,GlobalTravellors,5.0,Tourist attraction,https://lh5.googleusercontent.com/p/AF1QipOOxD...,0.012503
1,239,239,"XMM8+X48, 1st Cross Rd, Bhuvaneswari Nagar, C ...",Bengaluru,NaN,12.984916,77.665327,NaN,Karnataka,"XMM8+X48, 1st Cross Rd",Shiva Statue,4.7,"Tourist attraction, Hindu temple",https://lh5.googleusercontent.com/p/AF1QipP5XR...,0.014535
2,257,257,"2-6, 9th Cross Rd, Kaggadasapura, Bengaluru, K...",Bengaluru,NaN,12.985504,77.675954,NaN,Karnataka,"2-6, 9th Cross Rd",Chinimika layout,5.0,Tourist attraction,NaN,0.018832
3,234,234,"3D, No, 3, 5th Cross Rd, Michael Palaya, H Col...",Bengaluru,NaN,12.981284,77.646522,NaN,Karnataka,"3D, No, 3, 5th Cross Rd",Freedom Wall Bangalore,4.7,Tourist attraction,https://lh5.googleusercontent.com/p/AF1QipNt7G...,0.021202
4,236,236,"Police Station, Near HAL, HAL Old Airport Rd, ...",Bengaluru,Aerospace company's museum featuring photograp...,12.954655,77.680906,NaN,Karnataka,"Police Station, Near HAL, HAL Old Airport Rd",HAL Heritage Centre and Aerospace Museum,4.4,"Tourist attraction, Heritage museum, Museum",https://lh5.googleusercontent.com/p/AF1QipNb68...,0.022581


In [11]:

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer


# Define a function to calculate Jaccard similarity between a user's preferences and all restaurants
def calculate_similarity(user_id):
    user_prefs = user_preferences_df[user_preferences_df['user_id'] == user_id].iloc[:, 3:].values
    attraction_features = attractions_df.iloc[:, 5:].values

    # Create a MultiLabelBinarizer object to one-hot encode the values in each column as binary features
    mlb = MultiLabelBinarizer()

    # Fit the MultiLabelBinarizer on all the values in the 'Cuisines' column of the restaurants dataframe
    mlb.fit(attractions_df['categories'].str.split(', '))

    # Transform the 'Cuisines' column of both user preferences and restaurants dataframes into binary features
    user_categories = mlb.transform(user_preferences_df[user_preferences_df['user_id'] == user_id]['categories'].str.split(', '))
    attraction_categories = mlb.transform(attractions_df['categories'].str.split(', '))

    # Calculate Jaccard similarity between the user's preferences and all restaurants
    similarities = pairwise_distances(user_categories, attraction_categories, metric='jaccard')

    return 1 - similarities[0]





In [12]:
attraction_names = attractions_df['title']

In [13]:
# Example usage: Recommend top 5 restaurants for user 1
user_id = 1
similarities = calculate_similarity(user_id)
attraction_data = attractions_df[['title', 'categories', 'address', 'city', 'totalScore' ,'Distance']]
recommendations = pd.DataFrame({'title': attraction_names, 'Similarity': similarities})
recommendations = recommendations.merge(attraction_data, on='title').sort_values(by='Similarity', ascending=False).head(5)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [' Amusement center', ' Amusement park', ' Animal park', ' Art gallery', ' Auditorium', ' Bird watching area', ' Botanical garden', ' Cafe', ' Cathedral', ' Catholic church', " Children's museum", " Children's party service", ' Christian church', ' Church', ' Education center', ' Escape room center', ' Event management company', ' Event venue', ' Forestry service', ' Fortress', ' Garden', ' Heritage building', ' Heritage museum', ' Hindu temple', ' Historical landmark', ' Historical place', ' Historical place museum', ' Indoor snowcenter', ' Landmark', ' Learning center', ' Memorial park', ' Mosque', ' Museum', ' National forest', ' National museum', ' National park', ' Park', ' Park and Garden', ' Pet adoption service', ' Planetarium', ' Playground', ' Public beach', ' Research institute', ' Sculpture', ' Shopping mall', ' Software company', ' Souvenir store', ' Swimming pool', '

In [14]:
print(recommendations.to_string(index=False))
recommendations.head()

                title  Similarity                        categories                                                                                                     address      city  totalScore  Distance
           Car Park 🚗        0.25                              Park              VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout BTM 4th Stage, Bengaluru, Karnataka 560076, India Bengaluru         4.5  0.099294
Time For Travel Tours        0.20 Tour operator, Tourist attraction       No 734 Ground Floor, 1st B Cross Rd, 3rd Stage, Basaveshwar Nagar, Bengaluru, Karnataka 560079, India Bengaluru         5.0  0.128688
  Mahatma Gandhi Park        0.20          Tourist attraction, Park                                                 XHGX+VV9, Shivaji Nagar, Bengaluru, Karnataka 560001, India Bengaluru         4.1  0.065334
Buddha Shanthi Kanive        0.20          Tourist attraction, Park XGH7+VC9, 7th Cross Rd, 3rd Block, BDA Layout, 2nd Stage, Naagarabhaavi, Bengaluru, Karnataka 560072

,title,Similarity,categories,address,city,totalScore,Distance
125,Car Park 🚗,0.25,Park,"VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout B...",Bengaluru,4.5,0.099294
159,Time For Travel Tours,0.20,"Tour operator, Tourist attraction","No 734 Ground Floor, 1st B Cross Rd, 3rd Stage...",Bengaluru,5.0,0.128688
49,Mahatma Gandhi Park,0.20,"Tourist attraction, Park","XHGX+VV9, Shivaji Nagar, Bengaluru, Karnataka ...",Bengaluru,4.1,0.065334
179,Buddha Shanthi Kanive,0.20,"Tourist attraction, Park","XGH7+VC9, 7th Cross Rd, 3rd Block, BDA Layout,...",Bengaluru,4.4,0.151385
258,tannirubavi tree park,0.20,"Tourist attraction, Park","VRF9+WGP, Sultan Battery, Sultan Battery Road,...",Mangaluru,4.3,2.847485


In [17]:
# Example usage: Recommend top 5 restaurants for user 5
user_id = 7
similarities = calculate_similarity(user_id)
attraction_data = attractions_df[['title', 'categories', 'address', 'city', 'totalScore' ,'Distance']]
recommendations = pd.DataFrame({'title': attraction_names, 'Similarity': similarities})
recommendations = recommendations.merge(attraction_data, on='title').sort_values(by='Similarity', ascending=False).head(5)
print(recommendations.to_string(index=False))
recommendations.head()

                                 title  Similarity               categories                                                                                                               address      city  totalScore  Distance
                            Car Park 🚗    0.500000                     Park                        VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout BTM 4th Stage, Bengaluru, Karnataka 560076, India Bengaluru         4.5  0.099294
                        Childrens Park    0.333333 Tourist attraction, Park                                                    WJ7Q+CHQ, Sector 7, HSR Layout, Bengaluru, Karnataka 560102, India Bengaluru         4.0  0.062372
Arise Awake Park (formerly Lions Park)    0.333333 Tourist attraction, Park                                                            VRHW+F92, Bunts Hostel, Mangaluru, Karnataka 575003, India Mangaluru         4.2  2.820307
        Seetharampalya Lake View Point    0.333333 Tourist attraction, Park                     

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [' Amusement center', ' Amusement park', ' Botanical garden', ' Cafe', ' Catholic church', ' Chapel', " Children's museum", " Children's party service", ' Christian church', ' Church', ' Dog park', ' Education center', ' Escape room center', ' Event management company', ' Event venue', ' Forestry service', ' Heritage building', ' Heritage museum', ' Historical landmark', ' Historical place', ' Indoor snowcenter', ' Learning center', ' National forest', ' National park', ' Pet adoption service', ' Planetarium', ' Playground', ' Public library', ' Research institute', ' Science museum', ' Software company', ' Swimming lake', ' Swimming pool', ' Tourist attraction', ' Tourist information center'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.w

,title,Similarity,categories,address,city,totalScore,Distance
125,Car Park 🚗,0.500000,Park,"VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout B...",Bengaluru,4.5,0.099294
44,Childrens Park,0.333333,"Tourist attraction, Park","WJ7Q+CHQ, Sector 7, HSR Layout, Bengaluru, Kar...",Bengaluru,4.0,0.062372
231,Arise Awake Park (formerly Lions Park),0.333333,"Tourist attraction, Park","VRHW+F92, Bunts Hostel, Mangaluru, Karnataka 5...",Mangaluru,4.2,2.820307
27,Seetharampalya Lake View Point,0.333333,"Tourist attraction, Park","XPJ6+PX9, Seetharampalya, Hoodi, Bengaluru, Ka...",Bengaluru,3.9,0.049125
71,Indira gandhi Musical Fountain Park,0.333333,"Tourist attraction, Park","XHPR+4MV, Raj Bhavan Rd, Opposite to Jawahar L...",Bengaluru,4.1,0.074550


In [16]:
# Example usage: Recommend top 5 restaurants for user 20
user_id = 37
similarities = calculate_similarity(user_id)
attraction_data = attractions_df[['title', 'categories', 'address', 'city', 'totalScore' ,'Distance']]
recommendations = pd.DataFrame({'title': attraction_names, 'Similarity': similarities})
recommendations = recommendations.merge(attraction_data, on='title').sort_values(by='Similarity', ascending=False).head(5)
print(recommendations.to_string(index=False))
recommendations.head()

                  title  Similarity                          categories                                                                                                                                   address      city  totalScore  Distance
       GlobalTravellors         0.0                  Tourist attraction                                                                      Murugeshpalya, Jeevan Bima Nagar, Bengaluru, Karnataka 560017, India Bengaluru         5.0  0.012503
     Palanahalli Bridge         0.0                  Tourist attraction 4J85+WGC Airport Road, Palanahalli Rd, beside Jaguar Showroom international, Vinayak Nagar, Yelahanka, Bengaluru, Karnataka 560064, India Bengaluru         4.7  0.157242
           Stone quarry         0.0                  Tourist attraction                                                                 3HV3+4F8, Adityanagar, Vidyaranyapura, Bengaluru, Karnataka 560097, India Bengaluru         1.0  0.164693
       Turahalli Forest         

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [' Event management company'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,title,Similarity,categories,address,city,totalScore,Distance
0,GlobalTravellors,0.0,Tourist attraction,"Murugeshpalya, Jeevan Bima Nagar, Bengaluru, K...",Bengaluru,5.0,0.012503
187,Palanahalli Bridge,0.0,Tourist attraction,"4J85+WGC Airport Road, Palanahalli Rd, beside ...",Bengaluru,4.7,0.157242
193,Stone quarry,0.0,Tourist attraction,"3HV3+4F8, Adityanagar, Vidyaranyapura, Bengalu...",Bengaluru,1.0,0.164693
192,Turahalli Forest,0.0,"Tourist attraction, National forest","VGMF+FXQ, Thurahalli, Bengaluru, Karnataka 560...",Bengaluru,4.3,0.164438
191,Lakshmi Narsimha Temple,0.0,"Tourist attraction, Hindu temple","XGQ2+VXW, Sunkadakatte, Bengaluru, Karnataka 5...",Bengaluru,4.6,0.163408
